In [1]:
#Import libraries needed for the project
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout2D, ELU, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D, Conv2D
from keras.layers.core import Lambda
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
print("Importing Keras")
import numpy as np
print("Importing NumPy")
import csv
import cv2
print("Importing CSV and CV")
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle as sk_shuffle
print("Importing library to shuffle and split data")
from PIL import Image
print("Importing library to process data")
print("All set importing libraries...")

Using TensorFlow backend.


Importing Keras
Importing NumPy
Importing CSV and CV
Importing library to shuffle and split data
Importing library to process data
All set importing libraries...


In [2]:
# Project Variables
samples = []
X = []
y = []
X_train = []
y_train = []

# For model
nb_epoch = 30 # because of the version of TensorFlow library I am using variable name for epochs `nb_epochs`, it has been deprecated now
validation_split = 0.2 # Keep 20% of data for validatoin purpose

In [3]:
# Get CSV file path out of string
def csv_to_samples(csv_filepath):
    print("Open CSV file to load images data...")
    print("CSV file path", csv_filepath)
    samples = []
    with open(csv_filepath) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)
    # Pop the first item on the list to get rid of list headers
    removed_line = samples.pop(0)
    print("removing unused line", removed_line)
    return samples

In [4]:
def cols():
    # Number of columns in the resized image
    return 32

In [5]:
def rows():
    # Number of rows in the resized image
    return 11

In [6]:
def steering_correction():
    # How much to retate image on y axes if it comes from left or right camera
    return 0.275

In [7]:
def batch_size():
    return 128

In [8]:
def cropped_pixels():
    # how many pixels from the top to crop off of the image
    # these pixels should contain irrelevant to driving information
    # sky, trees, hirizon,...
    return 40

In [9]:
def random_brightness(image):
    """
    Returns an image with a random degree of brightness.
    :param image: Image represented as a numpy array.
    """
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    brightness = .25 + np.random.uniform()
    image[:,:,2] = image[:,:,2] * brightness
    image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
    return image

In [10]:
# Process image before converting into array
def pre_process_image(image, path_id, cols, rows):
    source_path = image[path_id]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    img = Image.open(current_path)
    img = np.array(img)
    
    # Crop only road part of the image as we do not need to train network on sky and 
    # other surraundings not related to the road and increase brightness
    img = random_brightness(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[cropped_pixels():,:,1]  
    # Resize all images to be same size to form matrixes of same shape
    return cv2.resize(img, (cols, rows))

In [11]:
def steering(arr):
    # data about steering is located in the array at position 3
    return arr[3]

In [12]:
# Generate array of data from CSV lines of samples
def generate_array(samples, batch_size):
    X, y = [], []
    print("Preprocessing images and Loading data...")
    for sample in samples:
        # central camera, no steering correction needed
        # path to the image from it is located in the array: sample at position 0
        X.append(pre_process_image(sample, 0, cols(), rows()))
        y.append(float(steering(sample)))

        # left camera, need to do steering correction from the right
        # path to the image from it is located in the array: sample at position 1
        X.append(pre_process_image(sample, 1, cols(), rows()))
        y.append(float(steering(sample)) + steering_correction())

        # right camera, need to do steering correction from the left
        # path to the image from it is located in the array: sample at position 2
        X.append(pre_process_image(sample, 2, cols(), rows()))
        y.append(float(steering(sample)) - steering_correction())

    X = np.array(X)
    y = np.array(y)
    print("Data Size: ", X.shape, y.shape)
    print("Loading data complete")
    return X, y

In [13]:
# Post process image array data
def post_process(X, y):
    # flip the image data horizontally and append the new data to the training data
    X = np.concatenate([X, X[:,:,::-1]])
    # negate the steering wheel angles of the flipped images
    y = np.concatenate([y, -y])

    # shuffle the training data
    X, y = sk_shuffle(X, y)

    X_train = X[:,:,:,None]
    y_train = y[:]

    print("Training Data Size: ", X_train.shape, y_train.shape)
    return X_train, y_train

In [14]:
# Get data for training ready
samples = csv_to_samples('data/driving_log.csv');
X, y = generate_array(samples, batch_size());
X_train, y_train = post_process(X=X, y=y);

Open CSV file to load images data...
CSV file path data/driving_log.csv
removing unused line ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
Preprocessing images and Loading data...
Data Size:  (24108, 11, 32) (24108,)
Loading data complete
Training Data Size:  (48216, 11, 32, 1) (48216,)


In [17]:
# Convolutional Neural Network Model Architecture
model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(rows(), cols(), 1), name='Normalization'))
model.add(Conv2D(2, 1, 1, border_mode='same', activation='relu'))
model.add(MaxPooling2D((4, 4), (4, 4), 'same'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(1))
# .summary function can provide data about the model input paramt and architecture, for reporting purposes
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Normalization (Lambda)           (None, 11, 32, 1)     0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 11, 32, 2)     4           Normalization[0][0]              
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 3, 8, 2)       0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 3, 8, 2)       0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

In [ ]:
from keras.callbacks import EarlyStopping
# Use Adam otimizer when compiling the model and mean square function to calculate loss
model.compile(
    loss='mean_squared_error',
    optimizer='adam'
)

# Train the model on the training data with 25 epochs
model.fit(
    X_train, 
    y_train, 
    batch_size=batch_size(), 
    verbose=1, 
    validation_split=validation_split, 
    nb_epoch=nb_epoch,
    shuffle=True
)

In [ ]:
# for the new drive.py file
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save("model.h5")
print("Saved model to disk")